In [13]:
# import libraries for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# import libraries for Tensorflow processing    
import numpy as np
import tensorflow as tf
import random
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
# import our chat-bot intents file
#from google.colab import files
#uploaded = files.upload()
with open('intent.json') as json_data:
    intents = json.load(json_data)
intents
    

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hello', 'Hey', 'Greetings', 'Good morning'],
   'responses': ['Hello! How can I help you?',
    'Hi there! What can I do for you today?']},
  {'tag': 'goodbye',
   'patterns': ['Goodbye', 'See you later', 'Bye', 'Farewell'],
   'responses': ['Goodbye! Have a great day.', 'See you later!']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', 'Appreciate it'],
   'responses': ["You're welcome!", 'No problem, happy to help!']},
  {'tag': 'unknown',
   'patterns': ['', "I don't know", 'Not sure'],
   'responses': ["I'm sorry, I didn't understand that.",
    'Could you please rephrase?']}]}

In [27]:
words=[]
classes=[]
documents=[]
ignore_words=['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w=nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w,intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [28]:
# perform steming and lower each word as well as remove duplicates
words=[stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))
# remove duplicates
classes=sorted(list(set(classes)))
print(len(documents),"documents")
print(len(classes),"classes",classes)
print(len(words),"unique stemmed words",words)

15 documents
4 classes ['goodbye', 'greeting', 'thanks', 'unknown']
21 unique stemmed words ['apprecy', 'bye', 'do', 'farewel', 'good', 'goodby', 'greet', 'hello', 'hey', 'hi', 'i', 'it', 'know', 'lat', 'morn', "n't", 'not', 'see', 'sur', 'thank', 'you']


In [29]:
# creating data for training
training=[]
output=[]
# create an empty array for our output
output_empty=[0]*len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag=[]
    # list of tokenized words for the pattern
    pattern_words=doc[0]
    # stem each word
    pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training=np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x=list(training[:,0])
train_y=list(training[:,1])
print("Training data created")

Training data created


C:\Users\HP\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [37]:
import tensorflow as tf

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))

# Use the new SGD optimizer without decay
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [38]:
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbot_model.h5',hist)
print("model created")

Epoch 1/200
3/3 [==============================] - 2s 20ms/step - loss: 1.4454 - accuracy: 0.1333
Epoch 2/200
3/3 [==============================] - 0s 13ms/step - loss: 1.3599 - accuracy: 0.3333
Epoch 3/200
3/3 [==============================] - 0s 9ms/step - loss: 1.3904 - accuracy: 0.2000
Epoch 4/200
3/3 [==============================] - 0s 7ms/step - loss: 1.3319 - accuracy: 0.3333
Epoch 5/200
3/3 [==============================] - 0s 13ms/step - loss: 1.3247 - accuracy: 0.2667
Epoch 6/200
3/3 [==============================] - 0s 10ms/step - loss: 1.2025 - accuracy: 0.5333
Epoch 7/200
3/3 [==============================] - 0s 13ms/step - loss: 1.2237 - accuracy: 0.5333
Epoch 8/200
3/3 [==============================] - 0s 13ms/step - loss: 1.2969 - accuracy: 0.4000
Epoch 9/200
3/3 [==============================] - 0s 9ms/step - loss: 1.2433 - accuracy: 0.3333
Epoch 10/200
3/3 [==============================] - 0s 15ms/step - loss: 1.1344 - accuracy: 0.6000
Epoch 11/200
3/3 [====

In [39]:
import pickle
pickle.dump({'words':words,'classes':classes,'train_x':train_x,'train_y':train_y},open("training_data","wb"))

In [40]:
# restore all of our data structures
data=pickle.load(open("training_data","rb"))
words=data['words']
classes=data['classes']

In [41]:
with open('intent.json') as json_data:
    intents = json.load(json_data)

In [42]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words=nltk.word_tokenize(sentence)
    # stem each word
    sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
def bow(sentence,words,show_details=False):
    # tokenize the pattern
    sentence_words=clean_up_sentence(sentence)
    # bag of words
    bag=[0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                bag[i]=1
                if show_details:
                    print("found in bag: %s" %w)
    return(np.array(bag))

In [43]:
ERROR_THRESHOLD=0.30
def classify(sentence):
    # generate probabilities from the model
    results=model.predict(np.array([bow(sentence,words)]))[0]
    # filter out predictions below a threshold
    results=[[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append((classes[r[0]],r[1]))
    # return tuple of intent and probability
    return return_list
def response(sentence,userID='123',show_details=False):
    results=classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag']==results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))
            results.pop(0)

In [45]:
response('Hi')

1/1 [==============================] - 0s 99ms/step
Hello! How can I help you?
